In [1]:
# Cleaning Data

# Create target object and call it y

# Create X

# Split into validation and training data

# Specify Model

# Fit Model

# Make validation predictions and calculate mean absolute error

# Show a prediction result and real data.

### Importing libraries.

Для анализа буду использовать стандартные библиотеки работы с данными, а так же библиотеки для построения графиков.

In [2]:
# Importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats

# Reduce the level of the axis logger to error:
# !!! Уберает все ненужные предупреждения, кроме ошибок.
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

### Reading the data.

In [3]:
# Impoirting the dataset
house_prices_dataset_path = "./data/house-prices-advanced-regression-techniques/sample_submission.csv"
house_prices_test_path = "./data/house-prices-advanced-regression-techniques/test.csv"
house_prices_train_path = "./data/house-prices-advanced-regression-techniques/train.csv"
house_data_discription_path = "./data/house-prices-advanced-regression-techniques/data_description.txt"

house_prices = pd.read_csv(house_prices_dataset_path)
house_prices_test = pd.read_csv(house_prices_test_path)
house_prices_train = pd.read_csv(house_prices_test_path)

In [4]:
# Exporting the data (name of columns) to .xlsx file to more convenient work.
# df = pd.DataFrame(house_prices_train.columns)
# df.to_excel('./auxiliary_files/show_columns.xlsx', sheet_name='Column Names')

#### Tips:

In [5]:
# How Concatanation several dataframes.

# clear_data = pd.concat([one_val_list, multi_val_list], axis=1)

# clear_data = pd.merge(missing_val_list,one_val_list,on='ID')

In [6]:
# How to filling missing values in the series.

# one_val_list[col_name]=one_val_list[col_name].fillna('None')
# one_val_list[col_name]=one_val_list[col_name].fillna(one_val_list[col_name].mean())

### Class

In [7]:
class Nan_counter:
    """Counting NaN(true) and NaN(false) values"""  
    
    def __init__(self, data_set):
        self.data = data_set

    def is_nan(self, col_name):
        """returning number of the NaN cells. In view True and False:
        True     1352
        False     107
        Name: column_name, dtype: int64"""
        return pd.isnull(self.data[self.col_name]).value_counts()
    
    def missing_values(self, accuracy):
        """accuracy in percentage
        #.missing_values(.01) - .01 is an accuracy in percent. 
        IOW, takes all values that less then .01 percent."""
        i = self.data.isnull().sum()
        j = len(self.data)
        y = round(i/j, 2)  
        return dict(y[y <= accuracy])

In [8]:
class Nan_columns:
    """
    - Finding the column(s) in dataframe that stored need number of NaN values,
    - Create a list with those columns"""
    
    def __init__(self, data_set):
        self.data = data_set
        
    def min_nan_number_in_column(self, min_required_nan):
        """finding the columns that stroed NaN where each column stored less or equal then # NaN values."""
        columns_of_dataset = self.data.columns.tolist()
        best_columns = []

        for i in columns_of_dataset:
            nan_sum = self.data[f"{i}"].isna().sum()            
            if nan_sum <= min_required_nan:
                best_columns.append(i)
        
        return best_columns
    
    def bad_column(self, val):
        """Select only those columns from the dataset with too many NaN rows.
           - `val` is percentage of bad rows"""
        columns_of_dataset = self.data.columns.tolist()
        bad_columns = []

        for i in columns_of_dataset:
            nan_sum = self.data[f"{i}"].isna().sum()            
            if val <= (nan_sum / len(self.data[f"{i}"])):
                bad_columns.append(i)
        
        return bad_columns

In [9]:
class Nan_string:
    """Show the string(s) that stored NaN value(s)"""
    
    def __init__(self, dataframe):
        self.df = dataframe
        
    def show_nan_string_in_dataframe(self):
        """Return only those strings that contain NaN values"""
        # creating and filling a dictionary (#str: col_name)
        nan_str_col = dict([(x, self.df.columns[y]) for x, y in zip(*np.where(self.df.isna()))])
        
        # takes only keys from dictionary
        keys = list(nan_str_col.keys())

        return self.df.loc[keys]
    
    def show_nan_string_in_column(self, col_name):
        """Showing # of NaN strings in each column."""
        count = 0 
        missing_str = []
        for i in house_prices_test[col_name]: 
            if i != i:
                missing_str.append(house_prices_test[col_name].index[count])
            count += 1
        return missing_str
        
    def show_nan_dict(self):
        """Return only number of the string and a name of the column that string(s) storing NaN values"""
        return dict([(x, self.df.columns[y]) for x, y in zip(*np.where(self.df.isna()))])
    


In [10]:
class Missing_column:
    """Differece between two lists(dataset and custom_sample.)"""
    
    def find_miss_col(self, col_set_main, col_set_test):
        """Showing the difference between two list"""
        return set(col_set_main) - set(col_set_test)

### Review Data (Understanding the Data)
#### First things first: analysing 'SalePrice'

In [11]:
# descriptive statistics summary
house_prices['SalePrice'].describe()

count      1459.000000
mean     179183.918243
std       16518.303051
min      135751.318893
25%      168703.011202
50%      179208.665698
75%      186789.409363
max      281643.976117
Name: SalePrice, dtype: float64

In [12]:
# Understanding the data component
house_prices['SalePrice'] = house_prices['SalePrice'].astype('int64')
full_dataset = pd.concat([house_prices_test, house_prices.drop(columns='Id')], axis=1)
full_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [13]:
full_dataset['YearBuilt'].value_counts(normalize = True, sort=True).reset_index()

,index,YearBuilt
0,2005,0.053461
1,2006,0.048663
2,2007,0.041124
3,2004,0.030843
4,2003,0.029472
...,...,...
101,1902,0.000685
102,1896,0.000685
103,1982,0.000685
104,1880,0.000685


In [14]:
# Showing only number columns.
full_dataset.select_dtypes(include=["int64", "float64"])

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,140,0,0,0,120,0,0,6,2010,169277
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,393,36,0,0,0,0,12500,6,2010,187758
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,212,34,0,0,0,0,0,3,2010,183583
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,360,36,0,0,0,0,0,6,2010,179317
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0,82,0,0,144,0,0,1,2010,150730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,...,0,0,0,0,0,0,0,6,2006,167081
1455,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,...,0,24,0,0,0,0,0,4,2006,164788
1456,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,...,474,0,0,0,0,0,0,9,2006,219222
1457,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,...,80,32,0,0,0,0,700,7,2006,184924


In [15]:
# Show all column that contain greater than ## missing values.
x = house_prices_test.isnull().sum()
x[x>4]

LotFrontage      227
Alley           1352
MasVnrType        16
MasVnrArea        15
BsmtQual          44
BsmtCond          45
BsmtExposure      44
BsmtFinType1      42
BsmtFinType2      42
FireplaceQu      730
GarageType        76
GarageYrBlt       78
GarageFinish      78
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
dtype: int64

In [16]:
# reviewing all columns and find number NaN strings inside.
# - takes all columns that NaN values(number of strings) are less than .## percent (ex: .01%/1459).
x = Nan_counter(house_prices_test).missing_values(.5)

# Converting all got result in a list.
features_01 = list(x)

print(features_01)
print('>>> Total:', len(features_01))

['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSol

In [17]:
# Select all columns which storing less or equal than # NaN strings.
features_02 = Nan_columns(house_prices_test).min_nan_number_in_column(4)

print(len(features_02))
print(features_02)

62
['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageCars', 'GarageArea', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']


In [18]:
# Select only those columns from the dataset with too many(greater than .## percent from total items (ex: .01/1459)) NaN rows.
# in this case, 1 percent = 14 missing values.
bad_data = Nan_columns(house_prices_test[features_01]).bad_column(.01)

print(len(bad_data))
print(bad_data)

14
['LotFrontage', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']


In [19]:
# finding the missing values columns from dataset to restructure them to improve the forcast accuracy.
missing_columns = Missing_column().find_miss_col(house_prices_test, bad_data)

print(list(house_prices_test[bad_data].isnull().sum()))
print(house_prices_test[bad_data].shape)

[227, 16, 15, 44, 45, 44, 42, 42, 730, 76, 78, 78, 78, 78]
(1459, 14)


In [20]:
# Finding the str_num and the col_name that storing NaN values.
nan_string = Nan_string(house_prices_test[bad_data])

# Observing those strings in DataFrame that storing NaN values. 
# String evaluation, visually analysis.
missing_val_list = pd.DataFrame(nan_string.show_nan_string_in_dataframe())
missing_val_list = missing_val_list.rename_axis('ID', index='columns')
missing_val_list

,LotFrontage,MasVnrType,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond
ID,,,,,,,,,,,,,,
0,80.0,None,0.0,TA,TA,No,Rec,LwQ,NaN,Attchd,1961.0,Unf,TA,TA
1,81.0,BrkFace,108.0,TA,TA,No,ALQ,Unf,NaN,Attchd,1958.0,Unf,TA,TA
4,43.0,None,0.0,Gd,TA,No,ALQ,Unf,NaN,Attchd,1992.0,RFn,TA,TA
6,NaN,None,0.0,Gd,TA,No,ALQ,Unf,NaN,Attchd,1992.0,Fin,TA,TA
9,70.0,None,0.0,TA,TA,No,ALQ,Rec,NaN,Attchd,1970.0,Fin,TA,TA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452,21.0,None,0.0,TA,TA,No,Rec,Unf,NaN,CarPort,1970.0,Unf,TA,TA
1453,21.0,None,0.0,TA,TA,No,Unf,Unf,NaN,NaN,NaN,NaN,NaN,NaN
1454,21.0,None,0.0,TA,TA,No,Unf,Unf,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# show # of NaN strings in each column.
for i in missing_val_list:
    print(i+":\n", Nan_string(missing_val_list).show_nan_string_in_column(f"{i}"))

LotFrontage:
 [6, 40, 41, 45, 47, 52, 59, 75, 82, 98, 103, 105, 107, 113, 119, 124, 132, 146, 152, 167, 174, 177, 179, 182, 183, 184, 187, 188, 199, 229, 230, 231, 235, 238, 240, 268, 271, 272, 273, 274, 276, 277, 279, 280, 283, 286, 290, 294, 297, 298, 301, 308, 359, 363, 373, 380, 383, 386, 387, 388, 401, 402, 403, 412, 418, 421, 423, 425, 442, 450, 451, 462, 476, 481, 485, 487, 489, 495, 497, 524, 525, 528, 529, 532, 536, 539, 563, 569, 570, 579, 581, 582, 584, 589, 592, 604, 614, 650, 651, 662, 668, 671, 677, 680, 681, 682, 686, 688, 695, 697, 703, 704, 706, 707, 710, 711, 713, 714, 715, 718, 726, 742, 743, 744, 762, 763, 774, 781, 785, 790, 793, 794, 797, 798, 808, 819, 840, 860, 865, 867, 897, 901, 912, 919, 927, 929, 935, 936, 943, 961, 963, 971, 986, 999, 1006, 1020, 1023, 1024, 1030, 1031, 1032, 1033, 1038, 1052, 1054, 1061, 1070, 1074, 1075, 1087, 1107, 1108, 1110, 1112, 1123, 1126, 1127, 1134, 1136, 1137, 1142, 1146, 1151, 1154, 1156, 1157, 1160, 1165, 1174, 1176, 1202, 1212

### Recognize, Fill or drop data

#### Hand Work (filling the miss value)

In [22]:
# show number of missing values for each column.
j = house_prices_test[bad_data].isnull().sum()
print(j)
print('>>> Total:', len(j))

LotFrontage     227
MasVnrType       16
MasVnrArea       15
BsmtQual         44
BsmtCond         45
BsmtExposure     44
BsmtFinType1     42
BsmtFinType2     42
FireplaceQu     730
GarageType       76
GarageYrBlt      78
GarageFinish     78
GarageQual       78
GarageCond       78
dtype: int64
>>> Total: 14


In [23]:
house_prices_test['GarageType'] = house_prices_test['GarageType'].fillna('NoData')
house_prices_test['BsmtFinType1'] = house_prices_test['BsmtFinType1'].fillna('NoData')
house_prices_test['BsmtFinType2'] = house_prices_test['BsmtFinType2'].fillna('NoData')
house_prices_test['BsmtExposure'] = house_prices_test['BsmtExposure'].fillna('NoData')
house_prices_test['BsmtCond'] = house_prices_test['BsmtCond'].fillna('NoData')
house_prices_test['MasVnrType'] = house_prices_test['MasVnrType'].fillna('NoData')
house_prices_test['MasVnrArea'] = house_prices_test['MasVnrArea'].fillna(house_prices_test['MasVnrArea'].mean())
house_prices_test['BsmtQual'] = house_prices_test['BsmtQual'].fillna('NoData')
house_prices_test['LotFrontage'] = house_prices_test['LotFrontage'].fillna(house_prices_test['LotFrontage'].mean())
house_prices_test['FireplaceQu'] = house_prices_test['FireplaceQu'].fillna('NoData')

In [24]:
df = house_prices_test

In [25]:
# GarageType: Garage location
col_name = 'GarageType'

print("The unique elements:\n")
print(df[col_name].value_counts())
print("\nThe Null values:", df[col_name].isnull().sum())

The unique elements:

Attchd     853
Detchd     392
BuiltIn     98
NoData      76
Basment     17
2Types      17
CarPort      6
Name: GarageType, dtype: int64

The Null values: 0


In [26]:
# BsmtFinType1: Rating of basement finished area
col_name = 'BsmtFinType1'

print("The unique elements:\n")
print(df[col_name].value_counts())
print("\nThe Null values:", df[col_name].isnull().sum())

The unique elements:

GLQ       431
Unf       421
ALQ       209
Rec       155
BLQ       121
LwQ        80
NoData     42
Name: BsmtFinType1, dtype: int64

The Null values: 0


In [27]:
# BsmtFinType2: Rating of basement finished area (if multiple types)
col_name = 'BsmtFinType2'

print("The unique elements:\n")
print(df[col_name].value_counts())
print("\nThe Null values:", df[col_name].isnull().sum())

The unique elements:

Unf       1237
Rec         51
NoData      42
LwQ         41
BLQ         35
ALQ         33
GLQ         20
Name: BsmtFinType2, dtype: int64

The Null values: 0


In [28]:
# BsmtExposure: Refers to walkout or garden level walls
col_name = 'BsmtExposure'

print("The unique elements:\n")
print(df[col_name].value_counts())
print("\nThe Null values:", df[col_name].isnull().sum())

The unique elements:

No        951
Av        197
Gd        142
Mn        125
NoData     44
Name: BsmtExposure, dtype: int64

The Null values: 0


In [29]:
# BsmtCond: Evaluates the general condition of the basement
col_name = 'BsmtCond'

print("The unique elements:\n")
print(df[col_name].value_counts())
print("\nThe Null values:", df[col_name].isnull().sum())

The unique elements:

TA        1295
Fa          59
Gd          57
NoData      45
Po           3
Name: BsmtCond, dtype: int64

The Null values: 0


In [30]:
# MasVnrType: Masonry veneer type
col_name = 'MasVnrType'

print("The unique elements:\n")
print(df[col_name].value_counts())
print("\nThe Null values:", df[col_name].isnull().sum())

The unique elements:

None       878
BrkFace    434
Stone      121
NoData      16
BrkCmn      10
Name: MasVnrType, dtype: int64

The Null values: 0


In [31]:
# MasVnrArea: Masonry veneer area in square feet
col_name = 'MasVnrArea'

print("The unique elements:\n")
print(df[col_name].value_counts())
print("\nThe Null values:", df[col_name].isnull().sum())

The unique elements:

0.000000      877
100.709141     15
176.000000     10
144.000000      9
120.000000      8
             ... 
634.000000      1
177.000000      1
615.000000      1
549.000000      1
342.000000      1
Name: MasVnrArea, Length: 304, dtype: int64

The Null values: 0


In [32]:
# BsmtQual: Evaluates the height of the basement
col_name = 'BsmtQual'

print("The unique elements:\n")
print(df[col_name].value_counts())
print("\nThe Null values:", df[col_name].isnull().sum())

The unique elements:

TA        634
Gd        591
Ex        137
Fa         53
NoData     44
Name: BsmtQual, dtype: int64

The Null values: 0


In [33]:
# LotFrontage: Linear feet of street connected to property
col_name = 'LotFrontage'

print("The unique elements:\n")
print(df[col_name].value_counts())
print("\nThe Null values:", df[col_name].isnull().sum())

The unique elements:

68.580357     227
60.000000     133
80.000000      68
70.000000      63
50.000000      60
             ... 
22.000000       1
136.000000      1
149.000000      1
31.000000       1
131.000000      1
Name: LotFrontage, Length: 116, dtype: int64

The Null values: 0


In [34]:
# FireplaceQu: Fireplace quality
col_name = 'FireplaceQu'

print("The unique elements:\n")
print(df[col_name].value_counts())
print("\nThe Null values:", df[col_name].isnull().sum())

The unique elements:

NoData    730
Gd        364
TA        279
Fa         41
Po         26
Ex         19
Name: FireplaceQu, dtype: int64

The Null values: 0


In [35]:
j = house_prices_test.isnull().sum()
j[j>4]

Alley           1352
GarageYrBlt       78
GarageFinish      78
GarageQual        78
GarageCond        78
PoolQC          1456
Fence           1169
MiscFeature     1408
dtype: int64

In [36]:
# Remove not useful columns.
col_name = ['Alley',
            'PoolQC',
            'Fence',
            'MiscFeature',
            'GarageYrBlt',
            'GarageFinish',
            'GarageQual',
            'GarageCond']

house_prices_test = house_prices_test.drop(columns=col_name)

In [37]:
j = house_prices_test.isnull().sum()
j[j>=0]

Id               0
MSSubClass       0
MSZoning         4
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         1
SaleCondition    0
Length: 72, dtype: int64

In [38]:
# Maked a clear dataset and add it a `SalePrice` column.
concat_sets = pd.concat([house_prices_test, house_prices], axis=1)

In [39]:
# Remove strings with NaN values.
clear_test_set = concat_sets.dropna()


# Checking the number of missing values.
print(f"Total missing values in DataFrame: {clear_test_set.isnull().sum().max()}")
print(f"Size of DF: {clear_test_set.shape}")

Total missing values in DataFrame: 0
Size of DF: (1447, 74)


### Building a Forcasting model

#### ToBeContinue > > > 
> Here I ran into a problem (I don't understand).  
  I need to tighten up the Mathematics.  
 *After completing the training by Data Analysis course*.